In [1]:
from __future__ import print_function
import argparse
import torch, os 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from feature_extractor.flickr import FLICKR
from feature_extractor.convvae import ConvVAE
import json
import csv

Output size torch.Size([64, 3, 500, 500])


In [2]:
# Training settings

batch_size = 64 #'input batch size for training (default: 64)')
test_batch_size = 1000 #'input batch size for testing (default: 1000)')
epochs = 10 #'number of epochs to train (default: 10)')
lr = 0.01 #'learning rate (default: 0.01)')
momentum = 0.5 #'SGD momentum (default: 0.5)')
cuda = False #'enables CUDA training')
seed = 1 #'random seed (default: 1)')
log_interval = 10 #'how many batches to wait before logging training status')

torch.manual_seed(seed)

if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [3]:
# Creating image_annotations

def retrieve_img_annotations(ID):

    # Opening JSON file
    f = open(f'./art_pieces_json/{ID}.json')
 
    # returns JSON object as
    # a dictionary
    data = json.load(f)

    # Closing file
    f.close()

    arkID = ID
    startYear = data['dateCreated'][0]['startYear']
    endYear = data['dateCreated'][0]['endYear']
    collection = data['collection']
    materialsAndTechniques = data['materialsAndTechniques']
    category = data['index']['Category'][0]['value']

    #print(arkID)

    try: 
        imagenery_value_1 = data['index']['Imagery'][0]['value']
    except:
        imagenery_value_1 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_2 = data['index']['Imagery'][1]['value']
    except:
        imagenery_value_2 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_3 = data['index']['Imagery'][2]['value']
    except:
        imagenery_value_3 = ''
        #print('no annotation \n')
        pass
        
    try: 
        imagenery_value_4 = data['index']['Imagery'][3]['value']
    except:
        imagenery_value_4 = ''
        #print('no annotation \n')
        pass
        
    try: 
        imagenery_value_5 = data['index']['Imagery'][4]['value']
    except:
        imagenery_value_5 = ''
        #print('no annotation \n')
        pass
        
    try: 
        imagenery_value_6 = data['index']['Imagery'][5]['value']
    except:
        imagenery_value_6 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_7 = data['index']['Imagery'][6]['value']
    except:
        imagenery_value_7 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_6 = data['index']['Imagery'][7]['value']
    except:
        imagenery_value_6 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_7 = data['index']['Imagery'][6]['value']
    except:
        imagenery_value_7 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_8 = data['index']['Imagery'][7]['value']
    except:
        imagenery_value_8 = ''
        #print('no annotation \n')
        pass

    try: 
        imagenery_value_9 = data['index']['Imagery'][8]['value']
    except:
        imagenery_value_9 = ''
        #print('no annotation \n')
        pass


    row = [arkID, startYear, endYear, collection, materialsAndTechniques, 
        category, imagenery_value_1, imagenery_value_2, imagenery_value_3, 
        imagenery_value_4, imagenery_value_5, imagenery_value_6, imagenery_value_7, 
        imagenery_value_8, imagenery_value_9]
    
    return row

In [6]:
# Saving image_annotations

row_list = []

for img_json in os.listdir('./art_pieces_json/'):
    #print(img_json)
    try:
        annotation_row = (retrieve_img_annotations(img_json[:-5]))
        row_list.append(annotation_row)
    except:
        print(f'{img_json} skipped.')
        pass

with open('./art_pieces_thumbnails/images_features/images_feat.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
        writer.writerows(row_list)  


cl010067011.json skipped.
cl020587021.json skipped.


In [ ]:
# Formatting image_annotations



In [7]:
# cnn_feature_extractor

import argparse
import os
import pickle
import pprint

from modules.utils import *


def convert_to_im_num(idx, array_num, max_per_array):
    return array_num * max_per_array + idx + 1


def create_feature_matrix(data, batch_size, model):
    data = np.transpose(data, (0, 3, 1, 2))
    print(data.shape)
    feats = []
    bs = batch_size
    start = 0
    end = start + bs
    while start < data.shape[0]:
        batch = convert_array_to_Variable(data[start:end])
        f = model(batch)
        feats.append(f)
        start = end
        end = start + bs
        if start % 50 == 0:
            print("Processed {} ims".format(start))
    feature_matrix = torch.cat(feats, 0)
    feature_matrix = feature_matrix.data.numpy()
    print(feature_matrix.shape)
    return feature_matrix

def create_feature_vector(data):
    model = load_model()
    feats = []
    data = np.transpose(data, (2, 0, 1))
    batch = convert_array_to_Variable(data)
    
    f = model(batch)
    feats.append(f)
    feature_matrix = torch.cat(feats, 0)
    feature_matrix = feature_matrix.data.numpy()
    print(feature_matrix.shape)
    return feature_matrix


def convert_to_dict(feature_matrix, max_per_array):
    feats = {}
    for i in range(feature_matrix.shape[0]):
        key = convert_to_im_num(i, 0, max_per_array)
        feature_vec = feature_matrix[i]
        feats[key] = feature_vec
    return feats

In [8]:
# cnn_feature_extractor

npy_path = 'data\images_numpy'
feat_path = 'data\\features'
batch_size = 10
model = load_model()
all_feats = []

for m in os.listdir(npy_path):
    if ".DS" in m:
        print("Skipping {}".format(m))
    else:
        print("Processing image array {}".format(m))
        m_num = str(m[:-4])
        #print(m_num)
        name = "feat_vec_" + m_num
        data = np.load(os.path.join(npy_path, m), allow_pickle=True)
        feature_matrix = create_feature_matrix(data, batch_size, model)
        np.save(os.path.join(feat_path, name), feature_matrix)
        feats_dict = convert_to_dict(feature_matrix, data.shape[0])
        for k in feats_dict:
            print(feats_dict[k].shape)
            break
        feats_with_names = {str(int(m_num) * 100 + k) + ".jpg": feats_dict[k] for k in feats_dict}
        pickle.dump(feats_with_names, open(os.path.join(feat_path, name + '.in'), "wb"))
        print("Saved feature matrix {}".format(m_num))
        keys = sorted(list(feats_with_names.keys()))
        print("Num keys: {}, first key: {}".format(len(keys), keys[0]))

        #
        # We don't need all feats because wont scale
        # changing the type to .p so that we can send it across stdin
        # making the keys filename so that we don't need to keep mapping it back
        #
        # all_feats.append(feature_matrix)
        # all_feats = np.concatenate(tuple(all_feats), axis=0)
        # print(all_feats.shape)
        # np.save(os.path.join(feat_path, "all_feat_vecs"), all_feats)
        # print("Saved all features in one matrix")
        # feats_dict = convert_to_dict(all_feats)
        # pickle.dump(feats_dict, open(os.path.join(feat_path, "feat_vec_dict.p"), 'wb'))
        # print("Saved feature dictionary")

Processing image array 0.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 0
Num keys: 1, first key: 1.jpg
Processing image array 1.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 1
Num keys: 1, first key: 101.jpg
Processing image array 10.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 10
Num keys: 1, first key: 1001.jpg
Processing image array 100.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 100
Num keys: 1, first key: 10001.jpg
Processing image array 101.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 101
Num keys: 1, first key: 10101.jpg
Processing image array 102.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 102
Num keys: 1, first key: 10201.jpg
Processing image array 103.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 103
Num keys: 1, first key: 10301.jpg
Processing image array 104.npy
(1, 3, 227, 227)
(1, 4096)
(4096,)
Saved feature matrix 104
Num keys: 1, first key: 10401.jpg
Processing image 

In [9]:
# Lists all keys in the feature vector pickle files
#check_key_conversion


# import pickle
# import os

# path = 'data\\features'
# for f in os.listdir(path):
#     if f.endswith('.in'):
#         data = pickle.load(open(os.path.join(path, f), 'rb'))
#         keys = sorted(list(data.keys()))
#         print("Keys for feature pickle file {}".format(f))
#         print(keys)

#     else:
#         pass

In [16]:
# Creating imageset

import os
import pandas as pd
from torchvision.io import read_image

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
train_loader = torch.utils.data.DataLoader(
    FLICKR(os.environ['dataset'], 
                   transform=transforms.Compose([
			transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:



kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    FLICKR(os.environ['dataset'], 
                   transform=transforms.Compose([
			transforms.ToTensor()
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
model = ConvVAE()

if args.cuda:
    model.cuda()